In [1]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.2 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp310-cp310-linux_x86_64.whl size=661265 sha256=f95df2e453c8bd66951d0c45f712e50c704d4af29bec80ccc628299a143548b5
  Stored in directory: /root/.cache/pip/wheels/1d/7b/06/82a395a243fce00035dea9914d92bbef0013401497d849f8bc
Successfully built pycuda


In [23]:
%%writefile Minmax_array.cu

#include <thrust/pair.h>
#include <thrust/device_vector.h>
#include <thrust/extrema.h>

// Wrapper function to call the Thrust function
extern "C" void my_cuda_function(float *d_a, float& minel, float& maxel, const int N) {

	thrust::pair<thrust::device_ptr<float>, thrust::device_ptr<float>> tuple;
	tuple = thrust::minmax_element(thrust::device_pointer_cast(d_a), thrust::device_pointer_cast(d_a) + N);
	minel = tuple.first[0];
	maxel = tuple.second[0]; }

Overwriting Minmax_array.cu


In [24]:
!nvcc -arch=sm_75 -o Minmax_array.so -shared -Xcompiler -fPIC Minmax_array.cu

In [28]:
import ctypes
import pycuda.gpuarray as gpuarray
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import pycuda.autoinit

import numpy as np

# Load the CUDA library
cuda_lib = ctypes.CDLL('./Minmax_array.so')  # Update with the correct path

minel = ctypes.c_float(0.)
maxel = ctypes.c_float(0.)

# Define the function prototype
cuda_lib.my_cuda_function.argtypes = [ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float), ctypes.c_int]
cuda_lib.my_cuda_function.restype = None

# Prepare data
input_data = np.array([3, 1, 33, -4]).astype(np.float32)
size = len(input_data)

# Use PyCUDA to allocate GPU memory
input_gpu   = gpuarray.to_gpu(input_data)

# Call the CUDA function
test = cuda_lib.my_cuda_function(ctypes.cast(input_gpu.ptr, ctypes.POINTER(ctypes.c_float)), ctypes.byref(minel), ctypes.byref(maxel), size)

print(input_gpu)
print(np.float32(minel))
print(np.float32(maxel))

[ 3.  1. 33. -4.]
-4.0
33.0
